In [1]:
import numpy as np
import pandas as pd
import statsmodels.api as sm
import seaborn as sns
import pickle

import matplotlib.pyplot as plt
plt.style.use("ggplot")
plt.style.use("dark_background")
%config InlineBackend.figure_format='retina'

import QuantTrading.ImpactFitting as IF



## Data Acquisition
Data initilialised in this section are all for all dates, all stocks

In [2]:
def load_from_pickle(filename):
    path = '../pkl_dump/'
    with open(path + filename, 'rb') as f:
        return pickle.load(f)

# Load data
traded_volume_df = load_from_pickle('traded_volume_df.pkl')
px_df = load_from_pickle('px_df.pkl')
daily_stock_info_df = load_from_pickle('daily_stock_info_df.pkl')
monthly_scaling_factor = load_from_pickle('monthly_scaling_factor.pkl')
stocks = traded_volume_df.reset_index()["stock"].unique()

## Price Impact (Unfinished)

There are several impact model to choose from: 1) naive OW 2) reduced form 3) AFS.

In [4]:
# time the code
import time
start = time.time()

half_life_list = [900, 1800, 3600, 7200]
rsq_data = np.zeros((len(stocks), len(half_life_list)))

for i in range(len(half_life_list)):
    model_type = "linear"
    impact_px_df = IF.get_impact_state(traded_volume_df, monthly_scaling_factor, 
                                    half_life_list[i], model_type)
    monthly_rsq = []
    for month in range(2, 12):
        reg_summary = IF.get_regression_results(impact_px_df, px_df, 
                                                in_sample_month=2, explanation_horizon_periods=6)
        rsq_data[:, i] += reg_summary[["is_rsq"]].values.flatten() / 10

rsq_table = pd.DataFrame(rsq_data, index=stocks, columns=half_life_list)

end = time.time()
print("Time taken: ", end - start)

/Users/kelvinwu/Library/CloudStorage/OneDrive-ImperialCollegeLondon/Year 4 MSc Maths and Finance/Quant Trading and Price Impact/QuantTradingProject/QuantTrading/ImpactFitting.py:28: FutureWarning: Support for axis=1 in DataFrame.ewm is deprecated and will be removed in a future version. Use obj.T.ewm(...) instead
  cum_impact = pre_ewm.ewm(alpha=1-decay_factor, adjust=False, axis="columns").mean()  # Across columns
/Users/kelvinwu/Library/CloudStorage/OneDrive-ImperialCollegeLondon/Year 4 MSc Maths and Finance/Quant Trading and Price Impact/QuantTradingProject/QuantTrading/ImpactFitting.py:28: FutureWarning: Support for axis=1 in DataFrame.ewm is deprecated and will be removed in a future version. Use obj.T.ewm(...) instead
  cum_impact = pre_ewm.ewm(alpha=1-decay_factor, adjust=False, axis="columns").mean()  # Across columns
/Users/kelvinwu/Library/CloudStorage/OneDrive-ImperialCollegeLondon/Year 4 MSc Maths and Finance/Quant Trading and Price Impact/QuantTradingProject/QuantTrading/I

Time taken:  1120.4359543323517


In [6]:
rsq_best = rsq_table.apply(lambda x: (x.idxmax(), x.max()), axis=1)
rsq_best

A        (1800, 0.05197246407805112)
AAL       (900, 0.08817299705311223)
AAP      (7200, 0.06874959604409892)
AAPL      (900, 0.13288244162703922)
ABBV     (1800, 0.07143555913349564)
ABC      (7200, 0.09752909263550626)
ABMD      (900, 0.07789431898994516)
ABT      (7200, 0.06633889358603762)
ACN      (3600, 0.04461016613730773)
ADBE     (1800, 0.04751666294577649)
ADI       (900, 0.04403971778412785)
ADM      (3600, 0.11165000814871719)
ADP      (900, 0.051869066503048684)
ADS      (7200, 0.06573949854930261)
ADSK     (7200, 0.07257164385707214)
AEE       (7200, 0.0359128652383136)
AEP      (1800, 0.03879433081569905)
AES      (7200, 0.04475744683597171)
AFL      (1800, 0.07839581904491953)
AGN       (900, 0.08077590650931665)
AIG      (1800, 0.09530727116484371)
AIV      (3600, 0.04327187180043712)
AIZ      (7200, 0.06116892372408243)
AJG      (7200, 0.06093239486357106)
AKAM     (1800, 0.06958354003794588)
ALB      (900, 0.057403988799959664)
ALGN    (7200, 0.027389056364180986)
A

## Synthetic Alpha (Done)

In [123]:
def synthetic_alpha_coef(corr, px_df_stock, ret_stock, alpha_horizon=6):
    ret_var = ret_stock.values.var()
    px_minus2 = (px_df_stock ** -2).values.mean()
    
    x = corr ** 2
    y = corr * np.sqrt(1 - corr ** 2) * np.sqrt(ret_var / alpha_horizon / px_minus2)
    return x, y

def diagnosis(returns_df):
    correlation = returns_df.corr().iloc[0,1]
    actual_variance = returns_df["actual"].var()
    synthetic_variance = returns_df["synthetic"].var()
    
    return correlation, actual_variance, synthetic_variance

def synthetic_alpha(corr, px_df, stock, alpha_horizon):
    px_df = px_df.loc[stock]
    ret = px_df.pct_change(alpha_horizon, axis="columns").dropna(axis=1)
    
    x, y = synthetic_alpha_coef(corr, px_df, ret, alpha_horizon)

    np.random.seed(42)
    
    returns = px_df.T.pct_change(alpha_horizon, axis=0).iloc[alpha_horizon:]
    returns.index.name = "time"

    px_changes = px_df.T.diff(alpha_horizon, axis=0).iloc[alpha_horizon:]
    W_diffs = np.random.normal(loc=0, scale=1.0, size=(px_df.shape[0], px_df.shape[1]-1))
    Ws = np.concatenate((np.zeros((W_diffs.shape[0], 1)), W_diffs.cumsum(axis=1)), axis=1).T
    W_h_diffs = Ws[alpha_horizon::] - Ws[:-alpha_horizon]
    px_changes = px_changes * x + W_h_diffs * y
    synthetic_returns = px_changes / (px_df.T.shift(1, axis=0))
    synthetic_returns.index.name = "time"

    returns_df = pd.DataFrame({
        "actual": returns.unstack(),
        "synthetic": synthetic_returns.unstack(),
    })
    
    print(diagnosis(returns_df))

    return returns_df

In [124]:
def get_synthetic_alpha(corr, px_df, stock, alpha_horizon=6, smooth=True):
    returns_df = synthetic_alpha(corr, px_df, stock, alpha_horizon)
    synthetic_alpha_diffs = returns_df.drop("actual", axis="columns").unstack("time")["synthetic"]
    
    synthetic_alphas = synthetic_alpha_diffs.iloc[:, ::-1].cumsum(axis="columns").iloc[:, ::-1].shift(-1, axis="columns").fillna(0)
    
    if smooth:
        return synthetic_alphas.ewm(halflife=200, axis="columns").mean()
    else:
        return synthetic_alphas

## Optimal Trading Strategy (Not started, should be very short)

## Backtesting (Barely Started)

In future versions use `cum_impacts` as input

In [168]:
def impact_adjusted_prices(pre_ewm, px_df, scaling_df, half_life, impact_coef_df, model_type):
    cum_impacts = impact_state(pre_ewm, scaling_df, half_life, model_type).T
    cum_returns = px_df.T / px_df.T.iloc[0, :] - 1
    stock_date_df = cum_returns.T.iloc[:, 0].reset_index()
    # stock_date_df["month"] = pd.to_datetime(stock_date_df["date"]).dt.month
    stock_date_df.drop(["date"], axis="columns", inplace=True)
    impact_coefficients = pd.merge(stock_date_df, impact_coef_df, on=["stock"], how="left")["beta_estimate"].values
    cum_returns -= cum_impacts * impact_coefficients
    adjusted_px_df = (px_df.T.iloc[0, :] * (cum_returns + 1)).T.reset_index()
    return adjusted_px_df


## Performance Analysis (Need plots!)